# Improting Data set

In [1]:
from datasets import load_dataset

C:\Users\Abdelsalam\anaconda3\envs\torchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (C:/Users/Abdelsalam/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 34.76it/s]


# Pre-Processing

In [3]:
from transformers import AutoTokenizer

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Abdelsalam\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-1b229fb576f6410a.arrow


# Reduce Number of data to speed up training and testing

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\Users\Abdelsalam\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-41a769da655b3614.arrow


# Traning

In [7]:
from transformers import AutoModelForSequenceClassification

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [9]:
from transformers import TrainingArguments

In [10]:
training_args = TrainingArguments(output_dir="test_trainer")

In [11]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer

In [14]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",num_train_epochs=2.0,
                                 per_device_train_batch_size = 4)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

C:\Users\Abdelsalam\anaconda3\envs\torchenv\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.182160,0.503000
2,1.154600,1.022104,0.599000


TrainOutput(global_step=500, training_loss=1.154553466796875, metrics={'train_runtime': 253.1875, 'train_samples_per_second': 7.899, 'train_steps_per_second': 1.975, 'total_flos': 526236284928000.0, 'train_loss': 1.154553466796875, 'epoch': 2.0})

In [62]:
# saving the tokenizer 
tokenizer.save_pretrained("./test_trainer/checkpoint-500/")

('./test_trainer/checkpoint-500/tokenizer_config.json',
 './test_trainer/checkpoint-500/special_tokens_map.json',
 './test_trainer/checkpoint-500/vocab.txt',
 './test_trainer/checkpoint-500/added_tokens.json',
 './test_trainer/checkpoint-500/tokenizer.json')

# Inference 

In [27]:
model = AutoModelForSequenceClassification.from_pretrained("./test_trainer/checkpoint-500/")
# tokenizer = AutoTokenizer.from_pretrained("./test_trainer/")

In [29]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [51]:
inputs = tokenizer("I can't express how bad I'm to do be able to use this stuoid product", return_tensors="pt")

# Perform inference
outputs = model(**inputs)

# Extract predictions from model outputs
logits = outputs.logits
predictions = logits.argmax(dim=-1)

In [52]:
logits

tensor([[ 2.5358,  1.8218, -0.1825, -1.6266, -2.1407]],
       grad_fn=<AddmmBackward0>)

In [53]:
predictions

tensor([0])

('./test_trainer/checkpoint-500/tokenizer_config.json',
 './test_trainer/checkpoint-500/special_tokens_map.json',
 './test_trainer/checkpoint-500/vocab.txt',
 './test_trainer/checkpoint-500/added_tokens.json',
 './test_trainer/checkpoint-500/tokenizer.json')

In [61]:
from transformers.convert_graph_to_onnx import convert
from pathlib import Path

model_name_or_path = "./test_trainer/checkpoint-500/"
onnx_model_path = Path("./test/bert-base-cased.onnx")

convert(framework="pt", model=model_name_or_path, output=onnx_model_path, opset=11)

ONNX opset version set to: 11
Loading pipeline (model: ./test_trainer/checkpoint-500/, tokenizer: ./test_trainer/checkpoint-500/)


Some weights of the model checkpoint at ./test_trainer/checkpoint-500/ were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Creating folder test
Using framework PyTorch: 1.13.0
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']
